# Import

In [77]:
import numpy as np
import pandas as pd
import csv
import datetime
import math
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb

# My Library

In [78]:
#製造answer.csv的第1列
def answer_header():
    header = []
    for i in range(28):
        header.append('time_slot_' + str(i) )
    return header

# Read File

In [79]:
# 訓練資料
reference = pd.read_csv('reference8.csv')
# 測試資料
answer_v2 = pd.read_csv('answer_v2.csv')
# 訓練期望輸出
the_labels = pd.read_csv('public/The_Labels.csv')

# Main

In [80]:
# 訓練資料
X = reference[answer_header()]
params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary',
          'nthread': 8,
          'num_leaves': 37,
          'learning_rate': 0.0479,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 0.889,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 0.14046,
          'reg_lambda': 1.20195,
          'min_split_gain': 0.5,
          'min_child_weight': 0.9998,
          'min_child_samples': 5,
          'scale_pos_weight': 1.30022,
          'num_class' : 1,
          'metric' : 'auc',
         }

y = []
model = []
# 訓練模型(28個 => time_slot0 ~ time_slot27)
for i in range(0,28):
    y.append(the_labels[answer_header()]['time_slot_' + str(i)])
    model.append(lgb.train(params, lgb.Dataset(X, y[i])))
    print(i)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [81]:
# 測試資料
data = answer_v2[answer_header()]
results = []
# 28個模型計算28行輸出
for i in range(28):
    result = model[i].predict(data)
    results.append(result)
    print(i)
# 寫入輸出csv
df = pd.DataFrame()
df['user_id'] = pd.Series(range(57159, 94251, 1))
for i in range(28):
    df['time_slot_' + str(i)] = pd.Series(results[i])
df.to_csv('answer_v7.csv', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [82]:
df

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,57159,0.787678,0.776576,0.564902,0.310585,0.771256,0.764463,0.493755,0.266422,0.665084,...,0.340262,0.204528,0.531453,0.573286,0.344096,0.317499,0.499525,0.653621,0.307071,0.132045
1,57160,0.109942,0.251557,0.045852,0.012586,0.329848,0.434305,0.085758,0.014056,0.168649,...,0.124708,0.008408,0.059731,0.189497,0.095145,0.010890,0.062478,0.169320,0.056874,0.012931
2,57161,0.023212,0.045716,0.008247,0.009072,0.025750,0.049808,0.006503,0.007740,0.026682,...,0.012924,0.015779,0.063347,0.075465,0.012779,0.018272,0.063221,0.069359,0.009506,0.008606
3,57162,0.029570,0.183942,0.097258,0.007287,0.051727,0.233869,0.075306,0.017376,0.033505,...,0.159770,0.024092,0.097406,0.149531,0.133234,0.015374,0.083922,0.256072,0.075161,0.009934
4,57163,0.026146,0.032851,0.006186,0.005198,0.023685,0.035971,0.006451,0.004912,0.023539,...,0.008321,0.005310,0.025834,0.025694,0.009233,0.006249,0.024076,0.021481,0.008039,0.003387
5,57164,0.023890,0.046708,0.007270,0.010144,0.027265,0.045919,0.008850,0.008403,0.025182,...,0.013314,0.020695,0.048760,0.079392,0.017540,0.024675,0.064159,0.056486,0.013058,0.014749
6,57165,0.028220,0.203748,0.058847,0.051848,0.035451,0.229454,0.080418,0.035617,0.042996,...,0.100492,0.062849,0.173037,0.269503,0.158650,0.051003,0.185076,0.275322,0.110871,0.075401
7,57166,0.015334,0.154540,0.023497,0.006426,0.024155,0.308067,0.028718,0.008366,0.020306,...,0.092263,0.010272,0.065954,0.187686,0.070961,0.010901,0.079168,0.205691,0.034042,0.006551
8,57167,0.688167,0.868426,0.102042,0.032165,0.758513,0.841839,0.151322,0.027163,0.643249,...,0.219797,0.048041,0.598737,0.674744,0.186139,0.100834,0.514928,0.701954,0.114217,0.012049
9,57168,0.754901,0.786594,0.014642,0.637338,0.853297,0.651348,0.014511,0.592962,0.816840,...,0.012610,0.194518,0.532488,0.288989,0.019295,0.100481,0.284541,0.213030,0.011245,0.392620


# Check File

In [83]:
reference

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.007812,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000977,0.000000,0.000000,0.000977,0.000000,0.000000,0.000000
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.125000,0.500000,0.000000
3,3,0.000000,0.000488,0.000000,0.000000,0.000488,0.500488,0.000000,0.000000,0.500000,...,0.500000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.500244,0.000244,0.000000
4,4,0.468750,0.968750,0.000000,0.812500,0.968750,0.968750,0.000000,1.562500,0.968750,...,0.000000,0.328125,0.484375,0.484375,0.328125,0.062500,0.234375,0.484375,0.000000,0.546875
5,5,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000
6,6,0.000488,0.691284,0.000488,0.000610,0.063110,0.507690,0.000244,0.000000,0.000000,...,0.001099,0.125244,0.948608,0.572144,0.500000,1.071533,0.751831,0.813904,0.000000,0.000000
7,7,0.625000,0.562500,0.562500,0.000000,0.687500,0.625000,0.500000,0.000000,0.687500,...,0.625000,0.000000,0.656250,0.812500,0.812500,0.500000,0.500000,0.937500,0.812500,0.000000
8,8,0.000977,0.001953,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.250000


In [84]:
answer_v2

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,57159,8.435059e-01,8.431549e-01,0.085327,1.511230e-01,8.749390e-01,9.919138e-01,2.340708e-01,2.630615e-01,7.245483e-01,...,5.977173e-01,1.641235e-01,1.963501e-01,6.573582e-02,3.235817e-03,3.176880e-01,7.570801e-01,9.211426e-01,6.689758e-01,0.450195
1,57160,1.562500e-02,7.656250e-01,0.015625,0.000000e+00,4.687500e-02,7.968750e-01,1.562500e-02,0.000000e+00,6.718750e-01,...,2.343750e-02,0.000000e+00,0.000000e+00,1.562500e-01,1.562500e-02,0.000000e+00,0.000000e+00,1.484375e-01,2.578125e-01,0.000000
2,57161,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.250000e-01,1.250000e-01,0.000000e+00,2.500000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
3,57162,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,...,5.224609e-01,0.000000e+00,9.765625e-04,8.544922e-03,2.612305e-02,0.000000e+00,2.441406e-04,1.252441e-01,1.250000e-01,0.000000
4,57163,2.500000e-01,2.500000e-01,0.000000,0.000000e+00,2.500000e-01,2.500000e-01,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
5,57164,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,2.500000e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.250000e-01,0.000000e+00,1.250000e-01,0.000000e+00,0.000000
6,57165,5.000000e-01,5.810547e-01,0.125977,1.953125e-03,0.000000e+00,9.619141e-01,3.828125e-01,3.300781e-01,0.000000e+00,...,9.765625e-04,0.000000e+00,3.134766e-01,7.031250e-02,2.500000e-01,3.906250e-03,2.543945e-01,3.833008e-01,2.500000e-01,0.258301
7,57166,0.000000e+00,7.500000e-01,0.000000,0.000000e+00,0.000000e+00,7.500000e-01,0.000000e+00,0.000000e+00,0.000000e+00,...,2.500000e-01,0.000000e+00,0.000000e+00,2.500038e-01,0.000000e+00,0.000000e+00,1.250000e-01,3.750029e-01,2.500000e-01,0.000000
8,57167,5.165340e-01,8.944780e-01,0.014687,0.000000e+00,8.818300e-01,9.296415e-01,5.889923e-03,0.000000e+00,7.997682e-01,...,5.030560e-01,0.000000e+00,2.686578e-01,7.669503e-01,1.810086e-02,4.882812e-04,7.775584e-01,8.878070e-01,4.959181e-03,0.000000
9,57168,8.631876e-01,9.916055e-01,0.000000,9.611805e-01,9.918211e-01,9.291990e-01,0.000000e+00,1.125015e+00,8.749998e-01,...,0.000000e+00,4.884243e-03,2.085586e-01,1.471043e-02,0.000000e+00,5.079346e-01,9.088230e-02,6.246567e-05,0.000000e+00,1.300200


In [85]:
the_labels

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,0,1,0,0,0,1,0,0,0,...,1,1,1,1,1,0,1,0,0,0
7,7,1,1,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
answer_v7 = pd.read_csv('answer_v7.csv')
answer_v7

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,57159,0.787678,0.776576,0.564902,0.310585,0.771256,0.764463,0.493755,0.266422,0.665084,...,0.340262,0.204528,0.531453,0.573286,0.344096,0.317499,0.499525,0.653621,0.307071,0.132045
1,57160,0.109942,0.251557,0.045852,0.012586,0.329848,0.434305,0.085758,0.014056,0.168649,...,0.124708,0.008408,0.059731,0.189497,0.095145,0.010890,0.062478,0.169320,0.056874,0.012931
2,57161,0.023212,0.045716,0.008247,0.009072,0.025750,0.049808,0.006503,0.007740,0.026682,...,0.012924,0.015779,0.063347,0.075465,0.012779,0.018272,0.063221,0.069359,0.009506,0.008606
3,57162,0.029570,0.183942,0.097258,0.007287,0.051727,0.233869,0.075306,0.017376,0.033505,...,0.159770,0.024092,0.097406,0.149531,0.133234,0.015374,0.083922,0.256072,0.075161,0.009934
4,57163,0.026146,0.032851,0.006186,0.005198,0.023685,0.035971,0.006451,0.004912,0.023539,...,0.008321,0.005310,0.025834,0.025694,0.009233,0.006249,0.024076,0.021481,0.008039,0.003387
5,57164,0.023890,0.046708,0.007270,0.010144,0.027265,0.045919,0.008850,0.008403,0.025182,...,0.013314,0.020695,0.048760,0.079392,0.017540,0.024675,0.064159,0.056486,0.013058,0.014749
6,57165,0.028220,0.203748,0.058847,0.051848,0.035451,0.229454,0.080418,0.035617,0.042996,...,0.100492,0.062849,0.173037,0.269503,0.158650,0.051003,0.185076,0.275322,0.110871,0.075401
7,57166,0.015334,0.154540,0.023497,0.006426,0.024155,0.308067,0.028718,0.008366,0.020306,...,0.092263,0.010272,0.065954,0.187686,0.070961,0.010901,0.079168,0.205691,0.034042,0.006551
8,57167,0.688167,0.868426,0.102042,0.032165,0.758513,0.841839,0.151322,0.027163,0.643249,...,0.219797,0.048041,0.598737,0.674744,0.186139,0.100834,0.514928,0.701954,0.114217,0.012049
9,57168,0.754901,0.786594,0.014642,0.637338,0.853297,0.651348,0.014511,0.592962,0.816840,...,0.012610,0.194518,0.532488,0.288989,0.019295,0.100481,0.284541,0.213030,0.011245,0.392620
